In [2]:
from keras.models import Model
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Input, Embedding
from keras.preprocessing.sequence import pad_sequences
from collections import Counter
from keras.callbacks import ModelCheckpoint
import nltk
import numpy as np
import os
import zipfile
import sys
import urllib.request
from gensim.models import KeyedVectors

Using TensorFlow backend.
C:\Users\santh_000\Anaconda3\envs\tfp3.6\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [18]:
BATCH_SIZE = 64
NUM_EPOCHS = 50
HIDDEN_UNITS = 256
NUM_SAMPLES = 10000
MAX_VOCAB_SIZE = 10000
EMBEDDING_SIZE = 200
DATA_PATH = 'fra.txt'

In [4]:
target_counter = Counter()

GLOVE_MODEL = "glove.6B.100d.txt"
fr_w2v = KeyedVectors.load_word2vec_format('frWac_non_lem_no_postag_no_phrase_200_cbow_cut100.bin', binary=True)
WEIGHT_FILE_PATH = 'eng-to-fr-glove-weights.h5'


In [5]:
lines = open(DATA_PATH, 'rt', encoding='utf8').read().split('\n')
for line in lines[: min(NUM_SAMPLES, len(lines)-1)]:
    target_text,input_text  = line.split('\t')
    input_words = [w for w in nltk.word_tokenize(input_text.lower())]
    target_text = '\t' + target_text + '\n'
    for char in target_text:
        target_counter[char] += 1


In [6]:
input_text

"J'enflammai l'allumette."

In [7]:
target_text

'\tI lit the match.\n'

In [8]:
target_word2idx = dict()
for idx, word in enumerate(target_counter.most_common(MAX_VOCAB_SIZE)):
    #print(word)
    target_word2idx[word[0]] = idx

target_idx2word = dict([(idx, word) for word, idx in target_word2idx.items()])

num_decoder_tokens = len(target_idx2word)

In [11]:
unknown_emb = np.random.randn(EMBEDDING_SIZE)

In [12]:
encoder_max_seq_length = 0
decoder_max_seq_length = 0

input_texts_word2em = []


In [13]:
lines = open(DATA_PATH, 'rt', encoding='utf8').read().split('\n')
for line in lines[: min(NUM_SAMPLES, len(lines)-1)]:
    target_text,input_text = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_words = [w for w in nltk.word_tokenize(input_text.lower())]
    encoder_input_wids = []
    for w in input_words:
        em = unknown_emb
        if w in fr_w2v:
            em = fr_w2v[w]
        encoder_input_wids.append(em)
    input_texts_word2em.append(encoder_input_wids)
    encoder_max_seq_length = max(len(encoder_input_wids), encoder_max_seq_length)
    decoder_max_seq_length = max(len(target_text), decoder_max_seq_length)

In [15]:
encoder_input_data = pad_sequences(input_texts_word2em, encoder_max_seq_length)

In [16]:
decoder_target_data = np.zeros(shape=(NUM_SAMPLES, decoder_max_seq_length, num_decoder_tokens))
decoder_input_data = np.zeros(shape=(NUM_SAMPLES, decoder_max_seq_length, num_decoder_tokens))
lines = open(DATA_PATH, 'rt', encoding='utf8').read().split('\n')
for lineIdx, line in enumerate(lines[: min(NUM_SAMPLES, len(lines)-1)]):
    target_text,_  = line.split('\t')
    target_text = '\t' + target_text + '\n'
    for idx, char in enumerate(target_text):
        if char in target_word2idx:
            w2idx = target_word2idx[char]
            decoder_input_data[lineIdx, idx, w2idx] = 1
            if idx > 0:
                decoder_target_data[lineIdx, idx-1, w2idx] = 1


In [17]:
context = dict()
context['num_decoder_tokens'] = num_decoder_tokens
context['encoder_max_seq_length'] = encoder_max_seq_length
context['decoder_max_seq_length'] = decoder_max_seq_length


In [20]:
encoder_inputs = Input(shape=(None, EMBEDDING_SIZE), name='encoder_inputs')
encoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, name='encoder_lstm')
encoder_outputs, encoder_state_h, encoder_state_c = encoder_lstm(encoder_inputs)
encoder_states = [encoder_state_h, encoder_state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens), name='decoder_inputs')
decoder_lstm = LSTM(units=HIDDEN_UNITS, return_state=True, return_sequences=True, name='decoder_lstm')
decoder_outputs, decoder_state_h, decoder_state_c = decoder_lstm(decoder_inputs,
                                                                 initial_state=encoder_states)
decoder_dense = Dense(units=num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')



checkpoint = ModelCheckpoint(filepath=WEIGHT_FILE_PATH, save_best_only=True)
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
          verbose=1, validation_split=0.2, callbacks=[checkpoint])

model.save_weights(WEIGHT_FILE_PATH)

Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 38s 5ms/step - loss: 2.0658 - val_loss: 2.2923


C:\Users\santh_000\Anaconda3\envs\tfp3.6\lib\site-packages\keras\engine\topology.py:2368: UserWarning: Layer decoder_lstm was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'encoder_lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'encoder_lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


Epoch 2/50
8000/8000 [==============================] - 40s 5ms/step - loss: 1.6166 - val_loss: 1.8556
Epoch 3/50
8000/8000 [==============================] - 38s 5ms/step - loss: 1.3606 - val_loss: 1.6914
Epoch 4/50
8000/8000 [==============================] - 36s 5ms/step - loss: 1.2275 - val_loss: 1.6312
Epoch 5/50
8000/8000 [==============================] - 36s 4ms/step - loss: 1.1321 - val_loss: 1.5607
Epoch 6/50
8000/8000 [==============================] - 36s 4ms/step - loss: 1.0512 - val_loss: 1.4778
Epoch 7/50
8000/8000 [==============================] - 35s 4ms/step - loss: 0.9825 - val_loss: 1.4820
Epoch 8/50
8000/8000 [==============================] - 35s 4ms/step - loss: 0.9199 - val_loss: 1.4512
Epoch 9/50
8000/8000 [==============================] - 33s 4ms/step - loss: 0.8625 - val_loss: 1.4251
Epoch 10/50
8000/8000 [==============================] - 41s 5ms/step - loss: 0.8104 - val_loss: 1.4510
Epoch 11/50
8000/8000 [==============================] - 35s 4ms/step - 

In [21]:
# Encoder inference model
encoder_model_inf = Model(encoder_inputs, encoder_states)

# Decoder inference model

decoder_state_input_h = Input(shape=(HIDDEN_UNITS,))
decoder_state_input_c = Input(shape=(HIDDEN_UNITS,))
decoder_input_states = [decoder_state_input_h, decoder_state_input_c]

decoder_out, decoder_h, decoder_c = decoder_lstm(decoder_inputs, 
                                                 initial_state=decoder_input_states)

decoder_states = [decoder_h , decoder_c]

decoder_out = decoder_dense(decoder_out)

decoder_model_inf = Model(inputs=[decoder_inputs] + decoder_input_states,
                          outputs=[decoder_out] + decoder_states )

In [22]:
max_encoder_seq_length = context['encoder_max_seq_length']
max_decoder_seq_length = context['decoder_max_seq_length']
num_decoder_tokens = context['num_decoder_tokens']

In [27]:
def translate_lang(input_text):
        input_seq = []
        input_wids = []
        for word in nltk.word_tokenize(input_text.lower()):
            emb = unknown_emb
            if word in fr_w2v:
                emb = fr_w2v[word]
            input_wids.append(emb)
        input_seq.append(input_wids)
        input_seq = pad_sequences(input_seq, max_encoder_seq_length)
        states_value = encoder_model_inf.predict(input_seq)
        target_seq = np.zeros((1, 1,num_decoder_tokens))
        target_seq[0, 0, target_word2idx['\t']] = 1
        target_text = ''
        terminated = False
        while not terminated:
            output_tokens, h, c = decoder_model_inf.predict([target_seq] + states_value)

            sample_token_idx = np.argmax(output_tokens[0, -1, :])
            sample_word = target_idx2word[sample_token_idx]
            target_text += sample_word

            if sample_word == '\n' or len(target_text) >= max_decoder_seq_length:
                terminated = True

            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sample_token_idx] = 1

            states_value = [h, c]
        return target_text.strip()

In [28]:
print(translate_lang('Il est malade.'))

He is sick.
